In [5]:
# https://attack.mitre.org/resources/attack-data-and-tools/
# https://github.com/mitre-attack/mitreattack-python/tree/master/mitreattack/attackToExcel

In [1]:
import mitreattack.attackToExcel.attackToExcel as attackToExcel

attackToExcel.export("enterprise-attack-v15.1", "v15.1", "./data")

ModuleNotFoundError: No module named 'mitreattack'

In [2]:
# usage instructions
python attackToExcell.py -h

SyntaxError: invalid syntax (2556411007.py, line 2)

In [ ]:
import pandas as pd
import janitor as jn
import plotly.express as px

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# read ATT&CK data locally
attack_file = pd.ExcelFile('./data/enterprise-attack-v15.1.xlsx')

df_datasources = pd.read_excel(attack_file, sheet_name='datasources')
df_tactics = pd.read_excel(attack_file, sheet_name='tactics')
df_techniques = pd.read_excel(attack_file, sheet_name='techniques')
df_relationships = pd.read_excel(attack_file, sheet_name='relationships')
df_mitigations = pd.read_excel(attack_file, sheet_name='mitigations')
df_software = pd.read_excel(attack_file, sheet_name='software')
df_groups = pd.read_excel(attack_file, sheet_name='groups') 

In [ ]:
## # read ATT&CK data from url
# url_attack = 'https://attack.mitre.org/docs/enterprise-attack-v15.1/enterprise-attack-v15.1.xlsx'
# df_datasources = pd.read_excel(url_attack, sheet_name='datasources')
# df_tactics = pd.read_excel(url_attack, sheet_name='tactics')
# df_techniques = pd.read_excel(url_attack, sheet_name='techniques')
# df_relationships = pd.read_excel(url_attack, sheet_name='relationships')
# df_mitigations = pd.read_excel(url_attack, sheet_name='mitigations')
# df_software = pd.read_excel(url_attack, sheet_name='software')
# df_groups = pd.read_excel(url_attack, sheet_name='groups')

In [ ]:
df_datasources = jn.clean_names(df_datasources)
df_tactics = jn.clean_names(df_tactics)
df_techniques = jn.clean_names(df_techniques)
df_relationships = jn.clean_names(df_relationships)
df_mitigations = jn.clean_names(df_mitigations)
df_software = jn.clean_names(df_software)
df_groups = jn.clean_names(df_groups)

In [ ]:
# Convert string of data sources to a list of data sources
df_techniques['data_sources'] = df_techniques['data_sources'].str.split(",")
# Use Pandas explode function to expand the list of data sources to separate rows
df_techniques = df_techniques.explode('data_sources').reset_index(drop=True)
# Get the technique ID and data sources then drop duplicate rows; place the output in a new data frame called 'viz_data' that'll be used for our visualization.
viz_data = df_techniques[['data_sources','id']].drop_duplicates().groupby(['data_sources']).size().reset_index()
# Rename column to 'count' in preparation for visualization
viz_data.columns = viz_data.columns.map(str)
viz_data = viz_data.rename(columns={"0": "count"})

In [ ]:
fig_te_by_ds  = px.bar(viz_data.sort_values('count', ascending=False).head(50), x='data_sources', y='count', title='MITRE ATT&CK: Sub-Technique Count by Data Source (Top 25)', labels={'count':'Technique Count', 'data_sources':'Data Source'})
fig_te_by_ds.show()

In [ ]:
df_software_malware = df_software.loc[df_software['type'] == 'malware']

df_software_malware.describe()


In [ ]:
df_groups[df_groups['description'].str.contains("retail")]['description']

In [ ]:
df_groups_FIN7 = df_groups.loc[df_groups['name'] == 'FIN7']

df_groups_FIN7

In [ ]:
df_relationships_FIN7 = df_relationships.loc[df_relationships['source_name'] == 'FIN7']

df_relationships_FIN7

In [ ]:
df_relationships_FIN7.count()

In [ ]:
#------------------------------------------------------------------------------

In [ ]:
df_groups_APT28 = df_groups.loc[df_groups['name'] == 'APT28']

df_groups_APT28

In [ ]:
df_relationships_APT28 = df_relationships.loc[df_relationships['source_name'] == 'APT28']

df_relationships_APT28 = df_relationships_APT28.loc[df_relationships_APT28['target_type'] == 'software']

df_relationships_APT28

In [ ]:
df_relationships_APT28.count()

In [ ]:
import graphviz
f = graphviz.Digraph(filename = "output/attacktree.gv")

names = ["1","2"]
positions = [df_relationships_APT28['source_name'].iloc[0],
             df_relationships_APT28['target_name'].iloc[0]]
for name, position in zip (names, positions):
     f.node(name, position, shape = "record", style = "filled", color = "skyblue")
 
#Specify edges
f.edge("1","2", label=" uses");

f